In [1]:
%pip install numpy datasets Pillow

Note: you may need to restart the kernel to use updated packages.


# Implementation

In [2]:
import numpy as np

In [3]:
def softmax(x):
    x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return x / np.sum(x, axis=-1, keepdims=True)

def log_softmax(x):
    x = x - np.max(x, axis=-1, keepdims=True)
    return x - np.log(np.sum(np.exp(x), axis=-1, keepdims=True))

In [4]:
def loss_fn(logits, y):
    return -log_softmax(logits)[y]

def d_loss_fn(logits, y):
    p = softmax(logits)
    p[y] -= 1
    return p

In [5]:
def relu(x):
    return np.maximum(0, x)

def d_relu(x):
    return np.where(x > 0, 1, 0)

In [6]:
def init_params(in_size: int, out_size: int, hidden_sizes: list[int]):
    dims = [in_size] + hidden_sizes + [out_size]
    params = []
    for nin, nout in zip(dims[:-1], dims[1:]):
        # xavier uniform initialization for weight matrixes
        a = (6 / (nin + nout)) ** 0.5
        w = np.random.uniform(-a, a, size=[nin, nout])
        b = np.zeros(nout)
        params.append([w, b])
    return params

def forward(x, params):
    tape = [(None, x)]
    for w, b in params:
        z = x @ w + b
        x = relu(z)
        tape.append((z, x))
    return z, tape[:-1]

def backprop(x, y, params):
    logits, tape = forward(x, params)

    grad = []
    error = d_loss_fn(logits, y)
    for (z, a), (w, _) in zip(reversed(tape), reversed(params)):
        grad.append((error * a.reshape(-1, 1), error))
        if z is not None:
            error = error @ w.T
            error = error * d_relu(z)
    grad = list(reversed(grad))
    
    return grad

# Training on MNIST

In [7]:
def train_mnist():
    import datasets

    mnist = datasets.load_dataset("mnist")
    xtrain, ytrain = np.array(mnist["train"]["image"]).reshape(-1, 784) / 255.0, mnist["train"]["label"]
    xtest, ytest = np.array(mnist["test"]["image"]).reshape(-1, 784) / 255.0, mnist["test"]["label"]

    def compute_val_acc(params):
        val_correct = 0
        for x, y in zip(xtest, ytest):
            z, _ = forward(x, params)
            val_correct += np.argmax(z) == y
        return val_correct / len(xtest)

    params = init_params(784, 20, [32, 16])
    lr = 1e-3
    n_examples = 10000
    log_every_n_steps = 100

    for step in range(n_examples):
        # get random training example
        i = np.random.randint(len(xtrain))
        x, y = xtrain[i], ytrain[i]

        # compute gradient
        grad = backprop(x, y, params)

        # update the parameters
        for k in range(len(params)):
            params[k][0] -= lr * grad[k][0] 
            params[k][1] -= lr * grad[k][1]

        # log
        if step % log_every_n_steps == 0:
            print(f"step: {step} | acc: {compute_val_acc(params):.4f}")

train_mnist()

/Users/jay/code/backpropagation/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


step: 0 | acc: 0.0416
step: 100 | acc: 0.0720
step: 200 | acc: 0.0867
step: 300 | acc: 0.1102
step: 400 | acc: 0.1325
step: 500 | acc: 0.1475
step: 600 | acc: 0.1478
step: 700 | acc: 0.1516
step: 800 | acc: 0.1569
step: 900 | acc: 0.1607
step: 1000 | acc: 0.1648
step: 1100 | acc: 0.1729
step: 1200 | acc: 0.1760
step: 1300 | acc: 0.1842
step: 1400 | acc: 0.1783
step: 1500 | acc: 0.2049
step: 1600 | acc: 0.2198
step: 1700 | acc: 0.2284
step: 1800 | acc: 0.2839
step: 1900 | acc: 0.3148
step: 2000 | acc: 0.3635
step: 2100 | acc: 0.3722
step: 2200 | acc: 0.4194
step: 2300 | acc: 0.4488
step: 2400 | acc: 0.4823
step: 2500 | acc: 0.4696
step: 2600 | acc: 0.5475
step: 2700 | acc: 0.5767
step: 2800 | acc: 0.5687
step: 2900 | acc: 0.5949
step: 3000 | acc: 0.5344
step: 3100 | acc: 0.6423
step: 3200 | acc: 0.6110
step: 3300 | acc: 0.6774
step: 3400 | acc: 0.6481
step: 3500 | acc: 0.6719
step: 3600 | acc: 0.6516
step: 3700 | acc: 0.6714
step: 3800 | acc: 0.6874
step: 3900 | acc: 0.6449
step: 4000 |

# Verifying Backprop

We can verify that our backprop gives a correct by comparing it with a numerically computed approximation of the gradient, which we can obtain via (using a small value for $h$):

$$
f'(x) = \lim_{h\rightarrow 0} \frac{f(x + h) - f(x)}{h}
$$

In [9]:
import copy

def numerical(x, y, params, h=1e-8):
    compute_loss = lambda params: loss_fn(forward(x, params)[0], y)  # noqa: E731

    base_loss = compute_loss(params)

    grad = copy.deepcopy(params)
    for i in range(len(params)):
        for j in range(len(params[i])):
            for k in np.ndindex(params[i][j].shape):
                prev_value = params[i][j][k]
                params[i][j][k] += h
                grad[i][j][k] = (compute_loss(params) - base_loss) / h
                params[i][j][k] = prev_value

    return grad

def verify():
    x, y = np.random.randn(256), np.random.randint(4)

    params = init_params(256, 4, [128, 64, 32, 16, 8])
    agrad = backprop(x, y, params)
    ngrad = numerical(x, y, params)

    rel_diffs = []
    for (aw, ab), (bw, bb) in zip(agrad, ngrad):
        rel_diffs += (abs(aw - bw) / (abs(aw) + np.finfo(aw.dtype).smallest_subnormal)).flatten().tolist()
        rel_diffs += (abs(ab - bb) / (abs(bb) + np.finfo(aw.dtype).smallest_subnormal)).flatten().tolist()
    
    rel_diffs = np.array(rel_diffs)
    
    print("Max Relative Difference:", rel_diffs.max())
    print("Mean Relative Difference:", rel_diffs.mean())

verify()

Max Relative Difference: 0.04297993519788267
Mean Relative Difference: 1.0049954290594662e-05
